In [1]:
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
train_data_dir = r"C:\Users\Admin\Desktop\emotional detection\archive (1)\train"
validation_data_dir = r"C:\Users\Admin\Desktop\emotional detection\archive (1)\test"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, color_mode = 'grayscale',
    target_size = (48, 48),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode = 'grayscale',
    target_size = (48, 48),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True)

class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
img, label = train_generator.__next__()

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape = (48, 48, 1)))

model.add(Conv2D(64, kernel_size = (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary())

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 256)    │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    13,107,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,277,831 (50.65 MB)

 Trainable params: 13,277,831 (50.65 MB)

 Non-trainable params: 0 (0.00 B)

None


In [2]:
train_path = r"C:\Users\Admin\Desktop\emotional detection\archive (1)\train"
test_path =  r"C:\Users\Admin\Desktop\emotional detection\archive (1)\test"

num_train_imgs = 0
for root, dirs, files in os.walk(train_path):
    num_train_imgs +=len(files)

num_test_imgs = 0
for root, dirs, files in os.walk(test_path):
    num_test_imgs +=len(files)
print(num_train_imgs)
print(num_test_imgs)

# train the model
epochs = 30
history = model.fit(train_generator,
                    steps_per_epoch=num_train_imgs//32,
                   epochs=epochs,
                   validation_data = validation_generator,
                   validation_steps = num_test_imgs//32)
model.save('model_file.h5')

28709
7178
Epoch 1/30


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


897/897 ━━━━━━━━━━━━━━━━━━━━ 5168s 6s/step - accuracy: 0.2538 - loss: 1.8307 - val_accuracy: 0.3661 - val_loss: 1.6204
Epoch 2/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.3125 - loss: 1.6421 - val_accuracy: 0.5000 - val_loss: 1.5271
Epoch 3/30


C:\ProgramData\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


897/897 ━━━━━━━━━━━━━━━━━━━━ 263s 292ms/step - accuracy: 0.3463 - loss: 1.6496 - val_accuracy: 0.4540 - val_loss: 1.4009
Epoch 4/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 64us/step - accuracy: 0.3125 - loss: 1.5730 - val_accuracy: 0.5000 - val_loss: 1.4987
Epoch 5/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 161s 179ms/step - accuracy: 0.4268 - loss: 1.4876 - val_accuracy: 0.4784 - val_loss: 1.3489
Epoch 6/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/step - accuracy: 0.4688 - loss: 1.4416 - val_accuracy: 0.7000 - val_loss: 0.8520
Epoch 7/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 166s 185ms/step - accuracy: 0.4515 - loss: 1.4120 - val_accuracy: 0.5198 - val_loss: 1.2624
Epoch 8/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 45us/step - accuracy: 0.4062 - loss: 1.6876 - val_accuracy: 0.5000 - val_loss: 1.0363
Epoch 9/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 175s 195ms/step - accuracy: 0.4768 - loss: 1.3627 - val_accuracy: 0.5364 - val_loss: 1.2086
Epoch 10/30
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 69us/step - accuracy: 0.3750 - loss: 1.2696 - val_ac

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
#Load the trained model
model = load_model('model_file.h5')
#Class labels (same order used during training)
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
#Start video capture
cap = cv2.VideoCapture(0)
#Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
while True:
    ret, frame = cap.read()
    if not ret:
        break
#Convert to grayscale because your model expects grayscale input
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors=5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi = roi_gray.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = model.predict(roi, verbose=0)[0]
        label = class_labels[np.argmax(preds)]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_COMPLEX, 0.9, (255, 0, 0), 2)
    cv2.imshow("Real-Time Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()